In [1]:
# download resume dataset
import kagglehub
import shutil

download_path = kagglehub.dataset_download("shamimhasan8/resume-vs-job-description-matching-dataset")
target_folder = "../dataset"

shutil.move(download_path, target_folder)

/Users/kirtan_patel/Documents/SproutsAI/vent/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 927k/927k [00:00<00:00, 4.37MB/s]

Extracting files...


'../dataset/1'

In [2]:
import kagglehub

download_path = kagglehub.dataset_download("surendra365/recruitement-dataset")

shutil.move(download_path, target_folder)

100%|██████████| 1.58M/1.58M [00:00<00:00, 1.91MB/s]

Extracting files...


'../dataset/2'

In [1]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
import torch

# use gpu is possible
device = "mps" if torch.backends.mps.is_available() else "cpu"

# load model on CPU first (to avoid meta tensor error)
model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
model = model.to(device)

/Users/kirtan_patel/Documents/SproutsAI/vent/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
file_path = "../dataset/resume_job_matching_dataset.csv"
df = pd.read_csv(file_path)

# normalize similarity score (1-5) linearly from 0 to 1
df['normal_score'] = (df['match_score'] - 1) / 4

In [3]:
import pandas as pd
file_path2 = "../dataset/job_applicant_dataset.csv"
df2 = pd.read_csv(file_path2)

In [4]:
print(df2)

     Job Applicant Name  Age  Gender             Race   Ethnicity  \
0          Daisuke Mori   29    Male  Mongoloid/Asian  Vietnamese   
1        Taichi Shimizu   31    Male  Mongoloid/Asian    Filipino   
2          Sarah Martin   46  Female  White/Caucasian       Dutch   
3          Keith Hughes   43    Male    Negroid/Black   Caribbean   
4           James Davis   49    Male  White/Caucasian     English   
...                 ...  ...     ...              ...         ...   
9995      Jada Williams   30  Female    Negroid/Black    Ghanaian   
9996       Jaden Carter   52    Male    Negroid/Black    Nigerian   
9997         Mia Foster   25  Female  White/Caucasian      German   
9998       Stella Green   51  Female  White/Caucasian       Irish   
9999        Ryo Nishida   46    Male  Mongoloid/Asian        Thai   

                                                 Resume             Job Roles  \
0     Proficient in Injury Prevention, Motivation, N...         Fitness Coach   
1     Pro

In [5]:
# make sure values are from 0 - 1
import numpy as np
df['normal_score'].describe()
df['normal_score'] = np.where(df['normal_score'] >= 0.5, 1, 0)
print(df['normal_score'])

0       1
1       1
2       1
3       1
4       1
       ..
9995    1
9996    0
9997    0
9998    0
9999    1
Name: normal_score, Length: 10000, dtype: int64


In [6]:
#combine 2 datasets
df2_renamed = df2.rename(columns={
    'Best Match': 'normal_score',
    'Job Description': 'job_description',
    "Resume" : 'resume'
})

columns_to_keep = ['job_description', 'resume', 'normal_score']
df2_sub = df2_renamed[columns_to_keep].copy()

for col in df.columns:
    if col not in df2_sub.columns:
        df2_sub.loc[:, col] = pd.NA

# Now concatenate
df_combined = pd.concat([df, df2_sub], ignore_index=True, sort=False)

In [7]:
df_combined

,job_description,resume,match_score,normal_score
0,"Data Analyst needed with experience in SQL, Ex...","Experienced professional skilled in SQL, Power...",4,1
1,Data Scientist needed with experience in Stati...,"Experienced professional skilled in Python, De...",4,1
2,Software Engineer needed with experience in Sy...,"Experienced professional skilled in wait, Git,...",5,1
3,"ML Engineer needed with experience in Python, ...","Experienced professional skilled in return, De...",4,1
4,Software Engineer needed with experience in RE...,"Experienced professional skilled in REST APIs,...",5,1
...,...,...,...,...
19995,A Biomedical Engineer designs and develops med...,"Proficient in Biology, Regulatory Compliance, ...",<NA>,0
19996,A Teacher shapes the future of students by del...,"Proficient in Communication, Teamwork, Lesson ...",<NA>,0
19997,"Diagnose and treat illnesses, prescribe medica...","Proficient in Medical Terminology, Critical Th...",<NA>,0
19998,A Fitness Coach is responsible for helping cl...,"Proficient in Exercise Programming, Motivation...",<NA>,1


In [8]:
import re

def clean_text(text):
    text = text.lower()
    # collapse multiple spaces/newlines into a single space
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-z0-9\s.,\-+#/]', '', text)
    return text.strip()

In [9]:
df_combined['job_description'] = df_combined['job_description'].apply(clean_text)
df_combined['resume'] = df_combined['resume'].apply(clean_text)

In [10]:
from sentence_transformers import InputExample

# create inputexample objects
examples = []
for _, row in df_combined.iterrows():
    examples.append(
        InputExample(texts=[row['resume'], row['job_description']], label=row['normal_score'])
    )

print(examples[1])

<InputExample> label: 1, texts: experienced professional skilled in python, deep learning, mr, our, nlp, machine learning, hair, pandas. senior such scene prepare soon last remember. general word what worry week responsibility. attack half scene thought push be.; data scientist needed with experience in statistics, pandas, sql, machine learning, nlp, deep learning, python. unit resource none quickly second interesting really end. across price book similar song give. black seem reach deep.


In [12]:
from torch.utils.data import DataLoader
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import Dataset


train_dataloader = DataLoader(examples, shuffle=True, batch_size=8)
train_loss = CosineSimilarityLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=2,
    warmup_steps= len(train_dataloader) * 0.1,
)


Step,Training Loss
500,0.188100
1000,0.171300
1500,0.169200
2000,0.165100
2500,0.165600
3000,0.160600
3500,0.158700
4000,0.157600
4500,0.157700
5000,0.154900


In [15]:
model.save("/Users/kirtan_patel/Documents/SproutsAI/model/all_minilm_finetuned")